# Project - Group 15
Jennifer, Irene, Zac, Yi <br>
Stat 159

In [16]:
# Import necessary packages
import pandas as pd
import numpy as np
from opioidtools import utils

## Data

### Import Datasets

#### ToDo: Add descriptions for dataset source

In [28]:
# Opioid Overdose Deaths 1999-2014
opioid = pd.read_csv('data/Multiple Cause of Death 1999-2014 v1.1.csv')

In [29]:
#GDP 2010-2022
gdp = pd.read_csv('data/2010 to 2022 Gross Domestic Product_ All Industry Total by State (Millions of Dollars).csv')

### Data Cleaning & Preprocessing
#### Opioid Data

In [30]:
# drop rows with suppressed, unreliable, or na values
opioid = opioid[opioid['Deaths']!='Suppressed']
opioid = opioid[opioid['Crude Rate']!='Unreliable']
opioid = opioid.dropna()

In [31]:
# convert quantitative data to numeric data type
opioid['Deaths'] = utils.to_numeric(opioid, 'Deaths')
opioid['Crude Rate'] = utils.to_numeric(opioid, 'Crude Rate')

In [32]:
# select opioid data within 2010-2014 (years have data for both gdp & opioid)
opioid_range = opioid[(opioid['Year']<=2014) & (opioid['Year']>=2010)]

# group data by year and state;
opioid_annual = opioid_range.groupby(['Year', 'State'],as_index=False).mean()

/tmp/ipykernel_33853/1788903058.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  opioid_annual = opioid_range.groupby(['Year', 'State'],as_index=False).mean()


#### GDP Data

In [33]:
# Reset indices & rename columns
gdp = gdp.reset_index()
gdp.columns = gdp.iloc[0]
gdp = gdp.tail(-1).reset_index(drop=True).iloc[:,0:8]
gdp.rename(columns={'Region Name':'State','2010-01-01': '2010', '2011-01-01': '2011','2012-01-01': '2012', '2013-01-01': '2013', '2014-01-01': '2014'}, inplace=True)

In [34]:
# rearrange gdp dataframe so to have years column (so can merge with opioid df)
# repeat region rows for each year (5 times)
region_info = gdp.iloc[:,0:3] 
region_info = region_info.loc[np.repeat(region_info.index, 5)].reset_index(drop=True)
# change gdp columns to rows
years_gdp = gdp.iloc[:,3:8]
years_gdp_arr = ([])
for i in range(51):
    years_gdp_arr = np.append(years_gdp_arr,years_gdp.iloc[i,:].to_numpy())
years_gdp=pd.DataFrame(years_gdp_arr, columns =['GDP_0101'])
years=pd.DataFrame([2010,2011,2012,2013,2014]*51,columns =['Year'])
# concatenate into new dataframe
gdp_annual=pd.concat([years,region_info,years_gdp], axis=1, join='inner')

### Merging Dataframes
#### Note: reduced to 251 rows bc don't have opioid data for every state for each year

In [38]:
# merge datasets by year and state
gdp_opioid = pd.merge(gdp_annual, opioid_annual, on=['Year','State'])
gdp_opioid

,Year,Series ID,State,Region Code,GDP_0101,Deaths,Population,Crude Rate,Prescriptions Dispensed by US Retailers in that year (millions)
0,2010,ALNQGSP,Alabama,01,177249.15,194.0,4779736.0,4.1,210.0
1,2011,ALNQGSP,Alabama,01,183916.625,180.0,4802740.0,3.7,219.0
2,2012,ALNQGSP,Alabama,01,189245.525,175.0,4822023.0,3.6,217.0
3,2013,ALNQGSP,Alabama,01,194786.925,175.0,4833722.0,3.6,207.0
4,2014,ALNQGSP,Alabama,01,197406.85,282.0,4849377.0,5.8,196.0
...,...,...,...,...,...,...,...,...,...
246,2010,WYNQGSP,Wyoming,56,37680.725,49.0,563626.0,8.7,210.0
247,2011,WYNQGSP,Wyoming,56,39751.375,47.0,568158.0,8.3,219.0
248,2012,WYNQGSP,Wyoming,56,38855.4,47.0,576412.0,8.2,217.0
249,2013,WYNQGSP,Wyoming,56,39289.775,52.0,582658.0,8.9,207.0


In [39]:
gdp_opioid.to_csv("data/gdp_opioid.csv",index=False)